In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
words = open("data/text8.txt").read().split(" ")
words, word2freq = min_count_threshold(words, min_count=5)
vocab, word2int, int2word = build_vocab(words)
len(word2freq)

71291

In [3]:
def parseVec(file, delimiter, word2int):
    lines = open(file).readlines()
    vocab_size, embed_size = [int(s) for s in lines[0].split()]
#     print(vocab_size)
    vocab_size  = min(len(word2int), vocab_size)
    embeddings = np.zeros((vocab_size + 1, embed_size), dtype=np.float64)
    for i in range(1, vocab_size):
        try:
            line = lines[i][:-1].split(delimiter)
            word = line[0]
            if word in word2int:
                wordvec = np.array([np.float64(j) for j in line[1:] if j != ''])
                embeddings[word2int[word]] = wordvec
        except Exception as e:
#             print(lines[i])
            print(e)
    return embeddings

In [23]:
seq_emb = parseVec('results/txt8vec.txt', ' ', word2int)

In [24]:
model = gensim.models.Word2Vec(corpus_file="data/text8.txt", size=100, iter=0)

In [25]:
model.wv.init_sims()
for gindex in range(len(model.wv.index2word)):
    gword = model.wv.index2word[gindex]
    index = word2int[gword]
    embedding = seq_emb[index]
    model.wv.vectors_norm[gindex] = embedding

In [26]:
result = model.wv.accuracy('data/countries.txt', restrict_vocab=len(model.wv.vocab), case_insensitive=False)

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.evaluate_word_analogies() instead).
  """Entry point for launching an IPython kernel.
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [40]:
for i in range(len(result)):
    print(result[i]['section'], len(result[i]['correct']), len(result[i]['incorrect']))

capital-common-countries 0 0
capital-world 0 0
currency 0 0
city-in-state 0 0
family 145 275
gram1-adjective-to-adverb 25 967
gram2-opposite 16 740
gram3-comparative 322 1010
gram4-superlative 105 887
gram5-present-participle 96 960
gram6-nationality-adjective 0 0
gram7-past-tense 361 1199
gram8-plural 310 1022
gram9-plural-verbs 69 801
total 1449 7861


In [51]:
sem, syn = 0, 0
synt, semt =0 ,0
for i in range(len(result) - 1):
    total = len(result[i]['correct']) +len( result[i]['incorrect'])
    if total == 0:
        continue
    correct = len(result[i]['correct'])
    if 'gram' in result[i]['section']:
        syn += correct
        synt += total
    else:
        sem  += correct
        semt += total
sem = sem * 100 / semt
syn = syn * 100 / synt
print("Semantic: {0:.02f}%, Syntactic: {1:.02f}%".format(sem, syn))

Semantic: 34.52%, Syntactic: 14.67%
